### [IMPORTANT] This step need to be completed manually

A. Get the labels coordinates with any tool and place them inside `all_position.py` \
B. When the file is loaded these coordinates will be transform to YOLO format, \
C. Finally fill the `positions.py` file


### 1A. Create the `label_list.py`


In [ ]:
import os
from os import pardir
from os.path import join, abspath

DEBUG = 0

current = os.getcwd()
folderName = current.split("\\")[-1]
root = abspath(join(current, pardir))

path = current + "\\image_groups\\"

dir = os.listdir(path)
folders: list[str] = []
for file in dir:
    filepath = f"{path}\\{file}"
    if os.path.isdir(filepath):
        folders.append(file)

with open("label_list.py", "w") as file:
    file.write("LABELS = [")
    for label in folders:
        file.write(f"'{label}', ")
    file.write("]")

print(path) if DEBUG else 0
print(folders) if DEBUG else 0

### 1B. Update the `obj.data` with the labels length and `obj.names`


In [ ]:
from label_list import LABELS

original = ""
with open("data/obj.data", "r") as file:
    original = file.read()

with open("data/obj.data", "w") as file:
    lines = original.split("\n")
    lines[0] = f"classes = {len(folders)}"
    text = "\n".join(lines)
    file.writelines(text)

with open("./data/obj.names", "w") as file:
    text = "\n".join(LABELS)
    print(text)
    file.write("\n".join(LABELS))  # type: ignore

### 1.C Create `label` folder


In [ ]:
from positions import POSITIONS
from label_list import LABELS
import os

if not os.path.exists("label"):
    os.mkdir("label")

obj = {}

for folder in folders:
    path = f"{current}\\image_groups\\{folder}"
    dir = os.listdir(path)

    for file in dir:
        filepath = f"{path}\\{file}"
        if not os.path.isdir(filepath):
            try:
                obj[file].append(f"{folder}")  # type: ignore
            except KeyError:
                obj[file] = [f"{folder}"]  # type: ignore

array: list[str] = []
for filename, labels in obj.items():  # type: ignore
    filepath: str = f"{current}\\label\\{filename.replace('png', 'txt')}"  # type: ignore
    try:
        with open(filepath, "w") as file:
            for label in labels:  # type: ignore
                i = LABELS.index(label)  # type: ignore
                p = POSITIONS[label]["position"]
                text = f"{i} {p}"
                array.append(text)
                file.write(text + "\n")
    except:
        pass

print("\n".join(list(set(array))))

### 1C. Create `data/obj` folder with .txt files


In [ ]:
from shutil import copy

if not os.path.exists("data/obj"):
    os.mkdir("data/obj")

file_prefixes = [f.split(".")[0] for f in os.listdir("label") if f.endswith(".txt")]

for prefix in file_prefixes:
    copy(f"label/{prefix}.txt", f"data/obj/{prefix}.txt")
    copy(f"images/{prefix}.png", f"data/obj/{prefix}.png")

### 2. Create the files need to train the neural network


In [ ]:
from label_list import LABELS
import glob

with open("./cfg/yolov4-tiny-custom_template.cfg", "r") as file:
    cfg_content = file.read()
    cfg_content = cfg_content.replace("_CLASS_NUMBER_", str(len(LABELS)))  # type: ignore
    cfg_content = cfg_content.replace(
        "_NUMBER_OF_FILTERS_", str((len(LABELS) + 5) * 3)  # type: ignore
    )
    cfg_content = cfg_content.replace(
        "_MAX_BATCHES_", str(max(6000, len(LABELS) * 2000))  # type: ignore
    )

with open("./cfg/yolov4-tiny-custom.cfg", "w") as file:
    file.write(cfg_content)


current_dir = "data/obj"

# Percentage of images to be used for the test set
percentage_test = 10

# Create and/or truncate train.txt and test.txt
file_train = open("data/train.txt", "w")
file_test = open("data/test.txt", "w")

# Populate train.txt and test.txt
counter = 1
index_test = round(100 / percentage_test)

for pathAndFilename in glob.iglob(os.path.join(current_dir, "*.png")):
    print(pathAndFilename)
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test:

        counter = 1

        file_test.write("data/obj" + "/" + title + ".png" + "\n")
    else:
        file_train.write("data/obj" + "/" + title + ".png" + "\n")
        counter = counter + 1

### 3. Train the neural network


In [ ]:
from subprocess import Popen, PIPE

current = os.getcwd()
root = abspath(join(current, pardir))

exe = abspath(join(root, join("darknet", "darknet.exe")))

data = "data/obj.data"
config = "cfg/yolov4-tiny-custom.cfg"
conv_original = "conv/yolov4.29"
conv_37 = "conv/yolov4.aws.37"

process = Popen(
    [exe, "detector", "train", data, config, conv_37], stdout=PIPE, text=True
)
while True:
    output = process.stdout.readline().replace("\n", "")  # type: ignore
    if process.poll() is not None:
        break
    if output:
        print(output)
process.poll()

### 4. Test the neural network


In [2]:
%run test_label_jp.py

Result images folder:  e:\Documents\Python\YOLO\RESULT_IMAGES\NEURAL_NET_AWS
creating images with labels..........
Result images ready!!!


### 5 [SUGGESTED] Get pre-trained weights


In [ ]:
from subprocess import Popen, PIPE

current = os.getcwd()
root = abspath(join(current, pardir))

exe = abspath(join(root, join("darknet", "darknet.exe")))

layers = 37
config = "cfg/yolov4-tiny-custom.cfg"
weight = "weight/aws_4icons.weights"
conv = f"conv/yolov4.aws.{layers}"

# ./darknet partial cfg/extraction.cfg extraction.weights extraction.conv.weights 24

process = Popen(
    [exe, "partial", config, weight, conv, f"{layers}"],
    stdout=PIPE,
    stderr=PIPE,
    shell=True,
    text=True,
)  # type
out, err = process.communicate()
print(out)
print(err)